In [1]:
import pynq
from pynq import Overlay
from pynq import DefaultIP
from pynq import MMIO
import time
from operator import*
import matplotlib.pyplot as plt

# Load the SPI overlay
ol = Overlay("adc_dma.bit")

In [2]:
ol?

In [3]:
print(ol.ip_dict)

{'adc_0': {'fullpath': 'adc_0', 'type': 'spicer.local:user:adc:1.0', 'state': None, 'addr_range': 65536, 'phys_addr': 1136656384, 'mem_id': 'S00_AXI', 'gpio': {}, 'interrupts': {}, 'parameters': {'C_S00_AXI_ADDR_WIDTH': '4', 'C_S00_AXI_DATA_WIDTH': '32', 'Component_Name': 'adc_dma_adc_0_0', 'EDK_IPTYPE': 'PERIPHERAL', 'C_S00_AXI_BASEADDR': '0x43C00000', 'C_S00_AXI_HIGHADDR': '0x43C0FFFF'}, 'registers': {}, 'device': <pynq.pl_server.device.XlnkDevice object at 0xb03a8750>, 'driver': <class 'pynq.overlay.DefaultIP'>}}


In [4]:
# Access the IP dictionary to find the address of the axi_quad_spi_0
spi_ip = ol.ip_dict['adc_0']
spi_base_address = (spi_ip['phys_addr'])
spi_address_range = (spi_ip['addr_range']) 

print(spi_base_address) 


#spi_base_address = 0x41E00000
#spi_address_range= 0x1000
# Set up MMIO for AXI Quad SPI
spi_mmio = MMIO(spi_address_range, spi_address_range)
spi_mmio.debug = True

1136656384


In [5]:
adc_ip = ol.adc_0  # Assuming the IP block name is adc_0

In [10]:
print(adc_ip.read())

0


In [ ]:
from pynq import Overlay
from pynq import allocate
import numpy as np

# Load the overlay and initialize the IP block
overlay = Overlay("adc_dma.bit")
adc_ip = overlay.adc_0  # Assuming the IP block name is adc_0

# Define memory buffer for DMA transfer
dma_data_buffer = allocate(shape=(100,), dtype=np.uint32)

# Start data collection loop
try:
    while True:
        # Check if the IP has valid data ready for DMA
        if adc_ip.read(0x04):  # Assuming a control register at offset 0x00 for data readiness
            # Start reading data to DMA buffer
            for i in range(100):
                dma_data_buffer[i] = adc_ip.read(i * 4)  # Read each 32-bit data point
                
            # Print the collected data
            print("ADC Data:", dma_data_buffer)
            
        else:
            print("Data not ready, waiting...")
        
except KeyboardInterrupt:
    print("Data collection stopped.")
finally:
    dma_data_buffer.free()  # Free DMA buffer
